In [1]:
import pandas as pd 
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import date, time, datetime, timedelta 

In [2]:
#defining the transformation from month to number of season 
def transform_month_to_season(x): 
    if x=='January' or x=='February' or x=='December': 
        return 3 
    if x=='March' or x=='April' or x=='May': 
        return 0
    if x=='June' or x=='July' or x=='August':
        return 1
    if x=='September' or x=='October' or x=='November':
        return 2

In [3]:
Dataset = pd.read_csv("boston_2019.csv", sep=',')
#first we will reound the time for each hour to have a better sight 
#rounding time 
df = pd.DataFrame(Dataset['start_time'])
df['start_time'] = pd.to_datetime(df.start_time)

# round start_time values to next nearest hour
# name it datetime so that it can be merged with the dates of the weather dataset
df['date_time'] = df['start_time'].dt.round('60min')
df = df.drop('start_time', axis='columns')
df['user'] = Dataset['user_type']

df['hour'] = df['date_time'].dt.hour

#we will add to the data frame a new column calle day to see 
#on how many bikes on each day of the week were rented (not over the year) 
df['day'] = df['date_time'].dt.day_name()
df['day'] = pd.Categorical(df['day'], categories = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],ordered = True)

#add month column to see te month of the row
df['month'] = df['date_time'].dt.month_name()
#to make the months in order
df['month'] = pd.Categorical(df['month'], categories = ['January','February','March','April','May','June','July','August','September','October','November','December'],ordered = True)

#to check how many rents over the weeks on a year
df['week_of_the_year'] = df['date_time'].dt.isocalendar().week

#to check how many rents over the season 
df['season'] = df['month'].apply(transform_month_to_season)

In [5]:
kpi_csrents_week = pd.DataFrame(df.groupby(["date_time","user"]).size())
kpi_csrents_week = kpi_csrents_week.rename(columns={0: "rents"})

In [6]:
df = pd.merge(df, kpi_csrents_week, on = ["date_time","user"])

In [7]:
df = df.drop_duplicates(subset=['date_time','user'], keep='first')

In [8]:
df['hour_of_the_year'] = np.arange(len(df))

In [10]:
test = pd.DataFrame(df.groupby(["date_time","user"]).size())

In [12]:
test = test.reset_index()
test = test.drop_duplicates(subset=['date_time'], keep = 'first')
test["count"] = np.arange(len(test))
test = test.rename(columns={0: "test"})

In [13]:
del test["test"]

In [14]:
df = pd.merge(df,test, on = "date_time")

In [15]:
df.to_csv('LogiReg.csv')

,date_time,user_x,hour,day,month,week_of_the_year,season,rents,hour_of_the_year,index,user_y,count
0,2019-01-01 00:00:00,Subscriber,0,Tuesday,January,1,3,1,0,0,Subscriber,0
1,2019-01-01 01:00:00,Subscriber,1,Tuesday,January,1,3,12,1,1,Customer,1
2,2019-01-01 01:00:00,Customer,1,Tuesday,January,1,3,6,2,1,Customer,1
3,2019-01-01 02:00:00,Subscriber,2,Tuesday,January,1,3,11,3,3,Customer,2
4,2019-01-01 02:00:00,Customer,2,Tuesday,January,1,3,4,4,3,Customer,2
...,...,...,...,...,...,...,...,...,...,...,...,...
16888,2019-12-31 22:00:00,Subscriber,22,Tuesday,December,1,3,33,16888,16887,Customer,8699
16889,2019-12-31 23:00:00,Subscriber,23,Tuesday,December,1,3,23,16889,16889,Customer,8700
16890,2019-12-31 23:00:00,Customer,23,Tuesday,December,1,3,11,16890,16889,Customer,8700
16891,2020-01-01 00:00:00,Subscriber,0,Wednesday,January,1,3,21,16891,16891,Customer,8701
